# 🔎 Real-Time Bitcoin News Q&A with Haystack
This notebook loads Bitcoin news from Elasticsearch, sets up the Haystack pipeline, and answers user questions.


In [19]:
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import launch_es

# 🚀 Start Elasticsearch (only needed if not already running via Docker Compose)
launch_es()

# 📡 Connect to Elasticsearch Document Store
document_store = ElasticsearchDocumentStore(
    host="localhost",
    username="",
    password="",
    index="bitcoin-news"
)

# 🔍 Set up Retriever
retriever = BM25Retriever(document_store=document_store)

# 🤖 Set up Extractive Reader
reader = FARMReader(
    model_name_or_path="deepset/roberta-base-squad2",
    use_gpu=False
)

# 🔗 Build the Extractive QA Pipeline
pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

# ❓ Define Q&A function with safe output handling
def ask_question(query):
    prediction = pipeline.run(
        query=query,
        params={"Retriever": {"top_k": 5}, "Reader": {"top_k": 3}}
    )

    if not prediction["answers"]:
        print("⚠️ No answer found. Try rephrasing your question or adding more detailed documents.")
        return

    print("📜 Answer:", prediction["answers"][0].answer)

    print("\n📚 Sources:")
    for doc in prediction["documents"]:
        print(f"- {doc.meta.get('source', 'N/A')} | {doc.meta.get('url', '')}")

# 🧪 Example usage
ask_question("What caused Bitcoin’s price drop this week?")


⚠️ No answer found. Try rephrasing your question or adding more detailed documents.
